# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import re
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df.message
Y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [7]:
pipeline

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = pipeline.predict(x_test)

In [11]:
cols = list(Y.columns)

In [12]:
for col in cols:
    index = cols.index(col)
    print(col, ':')
    print(classification_report(y_test[col], y_pred[:,index], target_names=cols))
    print('----------------------------------------------------------------------')

related :
                        precision    recall  f1-score   support

               related       0.60      0.35      0.44      1587
               request       0.81      0.93      0.86      4912
                 offer       0.53      0.16      0.25        55

           avg / total       0.76      0.78      0.76      6554

----------------------------------------------------------------------
request :
                        precision    recall  f1-score   support

               related       0.89      0.98      0.93      5434
               request       0.82      0.41      0.54      1120

           avg / total       0.88      0.88      0.87      6554

----------------------------------------------------------------------
offer :
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6525
               request       0.00      0.00      0.00        29

           avg / total       0.99      1.00      0.99 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                        precision    recall  f1-score   support

               related       0.93      0.99      0.96      6043
               request       0.58      0.09      0.15       511

           avg / total       0.90      0.92      0.90      6554

----------------------------------------------------------------------
medical_products :
                        precision    recall  f1-score   support

               related       0.95      1.00      0.97      6227
               request       0.60      0.06      0.10       327

           avg / total       0.94      0.95      0.93      6554

----------------------------------------------------------------------
search_and_rescue :
                        precision    recall  f1-score   support

               related       0.97      1.00      0.99      6374
               request       0.45      0.03      0.05       180

           avg / total       0.96      0.97      0.96      6554

------------------------------------------

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6554

           avg / total       1.00      1.00      1.00      6554

----------------------------------------------------------------------
water :
                        precision    recall  f1-score   support

               related       0.95      1.00      0.97      6150
               request       0.82      0.22      0.35       404

           avg / total       0.94      0.95      0.93      6554

----------------------------------------------------------------------
food :
                        precision    recall  f1-score   support

               related       0.93      0.99      0.96      5823
               request       0.85      0.44      0.58       731

           avg / total       0.92      0.93      0.92      6554

----------------------------------------------------------------------
shelter :
                        precision    recall  f1-s

                        precision    recall  f1-score   support

               related       0.85      0.98      0.91      5254
               request       0.81      0.32      0.46      1300

           avg / total       0.85      0.85      0.82      6554

----------------------------------------------------------------------


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7b6947881048>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [18]:
parameters = {
    'clf__estimator__n_estimators': [20, 50, 100],
    'clf__estimator__min_samples_split': [2],
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=2)

cv.fit(x_train, y_train)
best_params = cv.best_params_
best_model = cv.best_estimator_

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, total= 2.4min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, total= 2.5min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, total= 2.5min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total= 5.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total= 5.5min
[CV] clf__estimator__min_samples_split=2,

[Parallel(n_jobs=2)]: Done   9 out of   9 | elapsed: 30.0min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
y_pred = best_model.predict(x_test)

In [20]:
for col in cols:
    index = cols.index(col)
    print(col, ':')
    print(classification_report(y_test[col], y_pred[:,index], target_names=cols))
    print('----------------------------------------------------------------------')

related :
                        precision    recall  f1-score   support

               related       0.74      0.28      0.41      1587
               request       0.80      0.97      0.88      4912
                 offer       0.53      0.15      0.23        55

           avg / total       0.78      0.79      0.76      6554

----------------------------------------------------------------------
request :
                        precision    recall  f1-score   support

               related       0.89      0.99      0.94      5434
               request       0.90      0.43      0.58      1120

           avg / total       0.89      0.89      0.88      6554

----------------------------------------------------------------------
offer :
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6525
               request       0.00      0.00      0.00        29

           avg / total       0.99      1.00      0.99 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                        precision    recall  f1-score   support

               related       0.92      1.00      0.96      6043
               request       0.56      0.03      0.06       511

           avg / total       0.90      0.92      0.89      6554

----------------------------------------------------------------------
medical_products :
                        precision    recall  f1-score   support

               related       0.95      1.00      0.98      6227
               request       0.80      0.04      0.07       327

           avg / total       0.94      0.95      0.93      6554

----------------------------------------------------------------------
search_and_rescue :
                        precision    recall  f1-score   support

               related       0.97      1.00      0.99      6374
               request       0.78      0.04      0.07       180

           avg / total       0.97      0.97      0.96      6554

------------------------------------------

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


                        precision    recall  f1-score   support

               related       0.95      1.00      0.98      6150
               request       0.89      0.27      0.42       404

           avg / total       0.95      0.95      0.94      6554

----------------------------------------------------------------------
food :
                        precision    recall  f1-score   support

               related       0.93      0.99      0.96      5823
               request       0.89      0.40      0.55       731

           avg / total       0.92      0.93      0.91      6554

----------------------------------------------------------------------
shelter :
                        precision    recall  f1-score   support

               related       0.93      1.00      0.96      5979
               request       0.88      0.25      0.39       575

           avg / total       0.93      0.93      0.91      6554

----------------------------------------------------------------

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [22]:
import pickle

# Assuming you have a trained model called 'model'
# and you want to export it as a pickle file

# Save the model as a pickle file
with open('model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.